| ISS Tracker | International Space Station Tracker  |
| ----------- | ----------- |
| Date | Feb 2023 |
| ISS API url | http://api.open-notify.org/ |
| XXX API url | XXX |

In [63]:
# libraries
import requests
from geopy.geocoders import Nominatim
import geocoder
from datetime import datetime
from datetime import timezone
import pytz

# config file(s) import
# from config import IP_GEOLOCATION_API_KEY # IMPORTANT - KNOWN ERROR - Quit VS if you adjust this

In [64]:
"""Get the current location of the ISS"""
# send a GET request to the ISS API endpoint
iss_loc_response = requests.get("http://api.open-notify.org/iss-now.json")
iss_loc_data = iss_loc_response.json()

# print the API parameters
print(iss_loc_data)

{'timestamp': 1676462928, 'iss_position': {'longitude': '-133.0995', 'latitude': '-20.1446'}, 'message': 'success'}


In [65]:
# extract the latitude and longitude for the current location of the ISS
iss_latitude = float(iss_loc_data['iss_position']['latitude']) # also, converts lat string to float
iss_longitude = float(iss_loc_data['iss_position']['longitude']) # also, converts long string to float

# print the current latitude and longitude for the ISS
print("Current ISS Location:")
print(f"Latitude: {iss_latitude}")
print(f"Longitude: {iss_longitude}")

Current ISS Location:
Latitude: -20.1446
Longitude: -133.0995


In [66]:
# get the lat and long for the user's location
geolocator = Nominatim(user_agent="geoapiExercises")
# city = "San Diego"
city = "Rye, NY" # cool 
location = geolocator.geocode(city)
user_lat = location.latitude # returns a float object
user_long = location.longitude # returns a float object

# print my current location based on the city variable
# print(location.latitude, location.longitude)
print(f"User Location: {city}")
print(f"{city} Latitude: {user_lat}")
print(f"{city} Longitude: {user_long}")

User Location: Rye, NY
Rye, NY Latitude: 40.9808209
Rye, NY Longitude: -73.684294


In [71]:
# determine an approximate location relative to my exact location
approx_geolocator = Nominatim(user_agent="my-app")
approx_user_location = approx_geolocator.reverse(f"{user_lat}, {user_long}")

# print approximate user location
print(approx_user_location.address) 

Rye City Hall, 1051, Boston Post Road, City of Rye, Westchester County, New York, 10580, United States


In [67]:
from math import sin, cos, sqrt, atan2, radians

def distance(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

# libraries
import requests
from geopy.geocoders import Nominatim
from datetime import datetime, timezone
import pytz

"""Get the current location of the ISS"""
# send a GET request to the ISS API endpoint
iss_loc_response = requests.get("http://api.open-notify.org/iss-now.json")
iss_loc_data = iss_loc_response.json()

# extract the latitude and longitude for the current location of the ISS
iss_latitude = float(iss_loc_data['iss_position']['latitude'])
iss_longitude = float(iss_loc_data['iss_position']['longitude'])

# get the lat and long for the user's location
geolocator = Nominatim(user_agent="geoapiExercises")
city = "Rye, NY"
location = geolocator.geocode(city)
user_lat = location.latitude
user_long = location.longitude

# calculate the next time the ISS will pass over the user's location within 1200 miles
iss_pass_response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")
iss_pass_data = iss_pass_response.json()

next_pass_time = None
for p in iss_pass_data['response']:
    # calculate the distance from the user's location to the ISS during this pass
    pass_latitude = float(p['latitude'])
    pass_longitude = float(p['longitude'])
    pass_distance = distance(user_lat, user_long, pass_latitude, pass_longitude)

    # check if the distance is within 1200 miles
    if pass_distance <= 1200:
        # calculate the local time of the pass
        pass_time = datetime.fromtimestamp(p['risetime'], tz=timezone.utc)
        pass_time = pass_time.astimezone(pytz.timezone('US/Eastern'))

        # set the next pass time if it hasn't been set yet or if this pass is sooner than the current next pass time
        if next_pass_time is None or pass_time < next_pass_time:
            next_pass_time = pass_time

if next_pass_time is not None:
    print(f"Next ISS pass within 1200 miles of {city}: {next_pass_time}")
else:
    print(f"No upcoming ISS passes within 1200 miles of {city}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [70]:
# libraries
import requests
from geopy.geocoders import Nominatim
from datetime import datetime, timezone
import pytz

"""Get the current location of the ISS"""
# send a GET request to the ISS API endpoint
iss_loc_response = requests.get("http://api.open-notify.org/iss-now.json")
iss_loc_data = iss_loc_response.json()

# extract the latitude and longitude for the current location of the ISS
iss_latitude = float(iss_loc_data['iss_position']['latitude'])
iss_longitude = float(iss_loc_data['iss_position']['longitude'])

# get the lat and long for the user's location
geolocator = Nominatim(user_agent="geoapiExercises")
city = "Rye, NY"
location = geolocator.geocode(city)
user_lat = location.latitude
user_long = location.longitude

# calculate the next time the ISS will pass over the user's location within 1200 miles
iss_pass_response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")

if iss_pass_response.status_code == 200:
    try:
        iss_pass_data = iss_pass_response.json()
    except ValueError as e:
        print("Error decoding JSON: " + str(e))
        print("Response content: " + iss_pass_response.content.decode())
    else:
        next_pass_time = None
        for p in iss_pass_data['response']:
            # calculate the distance from the user's location to the ISS during this pass
            pass_latitude = float(p['latitude'])
            pass_longitude = float(p['longitude'])
            pass_distance = distance(user_lat, user_long, pass_latitude, pass_longitude)

            # check if the distance is within 1200 miles
            if pass_distance <= 1200:
                # calculate the local time of the pass
                pass_time = datetime.fromtimestamp(p['risetime'], tz=timezone.utc)
                pass_time = pass_time.astimezone(pytz.timezone('US/Eastern'))

                # set the next pass time if it hasn't been set yet or if this pass is sooner than the current next pass time
                if next_pass_time is None or pass_time < next_pass_time:
                    next_pass_time = pass_time

        if next_pass_time is not None:
            print(f"Next ISS pass within 1200 miles of {city}: {next_pass_time}")
        else:
            print(f"No upcoming ISS passes within 1200 miles of {city}")
else:
    print("API Error: " + iss_pass_response.reason)


API Error: Not Found


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def distance(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

In [ ]:
# calculate the next time the ISS will pass over the user's location within 1200 miles
iss_pass_response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")
iss_pass_data = iss_pass_response.json()

next_pass_time = None
for p in iss_pass_data['response']:
    # calculate the distance from the user's location to the ISS during this pass
    pass_latitude = float(p['latitude'])
    pass_longitude = float(p['longitude'])
    pass_distance = distance(user_lat, user_long, pass_latitude, pass_longitude)

    # check if the distance is within 1200 miles
    if pass_distance <= 1200:
        # calculate the local time of the pass
        pass_time = datetime.fromtimestamp(p['risetime'], tz=timezone.utc)
        pass_time = pass_time.astimezone(pytz.timezone('US/Eastern'))

        # set the next pass time if it hasn't been set yet or if this pass is sooner than the current next pass time
        if next_pass_time is None or pass_time < next_pass_time:
            next_pass_time = pass_time

if next_pass_time is not None:
    print(f"Next ISS pass within 1200 miles of {city}: {next_pass_time}")
else:
    print(f"No upcoming ISS passes within 1200 miles of {city}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# calculate the next time the ISS will pass over the user's location
iss_pass_response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")

if iss_pass_response.status_code == 200:
    iss_pass_data = iss_pass_response.json()
    if iss_pass_data['message'] == 'success':
        next_pass_time = datetime.fromtimestamp(iss_pass_data['response'][0]['risetime'], tz=timezone.utc)
        next_pass_time = next_pass_time.astimezone(pytz.timezone('US/Eastern'))
        print(f"Next ISS pass over {city}: {next_pass_time}")
    else:
        print("API Error: " + iss_pass_data['reason'])
else:
    print("API Error: " + iss_pass_response.reason)

API Error: Not Found


In [ ]:
# calculate the next time the ISS will pass over the user's location
iss_pass_response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")
iss_pass_data = iss_pass_response.json()
next_pass_time = datetime.fromtimestamp(iss_pass_data['response'][0]['risetime'], tz=timezone.utc)
next_pass_time = next_pass_time.astimezone(pytz.timezone('US/Eastern'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# GET the current timestamp in UTC
timestamp = datetime.utcfromtimestamp(iss_loc_data["timestamp"])

# make a request to the Open Notify API to get the next pass of the ISS over the user location
response = requests.get(f"http://api.open-notify.org/iss-pass.json?lat={user_lat}&lon={user_long}")
iss_next_pass = response.json()

| END OF PROGRAM | 
| ----------- |
| NOTES BELOW |

In [ ]:
# get the location of my ip address
g = geocoder.ip('me')
my_latitude, my_longitude = g.latlng
print("Latitude:", my_latitude) # float
print("Longitude:", my_longitude) # float

Latitude: 41.0018
Longitude: -73.6657


In [ ]:
# get my location geocoded aka reverse coded address
geolocator = Nominatim(user_agent="my-app")
my_location = geolocator.reverse(f"{my_latitude}, {my_longitude}")

# print a location near to where I am -> close enough at least for ISS viewing
# print(my_location.address) 

In [ ]:
"""Get the latitude and longitude of the users location"""
# get my devices public ip address
device_public_ip_response = requests.get("http://icanhazip.com")
device_public_ip = device_public_ip_response.text.strip()

# print(f"My Public IP Address: {device_public_ip}")

device_public_ip

'24.188.168.62'

In [ ]:
response = requests.get(f"https://ipgeolocation.io/ipgeo?apiKey={IP_GEOLOCATION_API_KEY}&ip={device_public_ip}")
data = response.json()

# latitude = data["latitude"]
# longitude = data["longitude"]

# print(f"Latitude: {latitude}")
# print(f"Longitude: {longitude}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# get my location
device_loc_response = requests.get(f"https://ipgeolocation.io/ipgeo?apiKey={IP_GEOLOCATION_API_KEY}&ip='24.188.168.62'")
device_loc_data = device_loc_response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# convert lat/long strings to float objects
iss_latitude_float = float(iss_latitude_string)
iss_longitude_float = float(iss_longitude_string)

# print lat/long as float objects
print(iss_latitude_float)
print(iss_longitude_float)

-45.8932
14.2831


In [ ]:
"""Get the latitude and longitude of the users location"""

ip_address = "8.8.8.8"

response = requests.get(f"https://ipgeolocation.io/ipgeo?apiKey={IP_GEOLOCATION_API_KEY}&ip={ip_address}")
data = response.json()

latitude = data["latitude"]
longitude = data["longitude"]

print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
"""Get the latitude and longitude of the users location"""
# GET the user's location
user_loc_response = requests.get(f"https://api.ipgeolocation.io/ipgeo?apiKey={IP_GEOLOCATION_API_KEY}")
user_loc_data = user_loc_response.json()
print(user_loc_data)

# # extract the latitude and longitude from the response
# user_latitude = float(user_loc_data["latitude"])
# user_longitude = float(user_loc_data["longitude"])

# # print the latitude and longitude of the users location
# print(f"User's Latitude: {user_latitude}")
# print(f"User's Longitude: {user_longitude}")



{'message': 'Provided API key is not valid. Contact technical support for assistance at support@ipgeolocation.io'}


In [ ]:
# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Define the latitude and longitude
latitude = -32.5395
longitude = -75.9591

# Use the geolocator to reverse geocode the location
location = geolocator.reverse(f"{latitude}, {longitude}")

if location is not None:
    # Extract the address components from the location
    address = location.raw["address"]
    country = address.get("country", "")
    ocean = address.get("ocean", "")
    sea = address.get("sea", "")

    # Print the results
    if country:
        print(f"Country: {country}")
    if ocean:
        print(f"Ocean: {ocean}")
    if sea:
        print(f"Sea: {sea}")
else:
    print("Location not found.")


Location not found.


In [ ]:
latitude = iss_latitude_float
longitude = iss_longitude_float

In [ ]:
geolocator = Nominatim(user_agent="my-app")
latitude = -32.5395
longitude = iss_longitude_float

location = geolocator.reverse(f'{latitude}, {longitude}')
print(location)

# extract address components from the ISS location
# address = iss_proximity_location.raw["address"]
# country = address.get("country", "")
# ocean = address.get("ocean", "")
# sea = address.get("sea", "")

# if country:
#     print(f"Country: {country}")
# if ocean:
#     print(f"Ocean: {ocean}")
# if sea:
#     print(f"Sea: {sea}")
# else:
#     print("Location not found.")



None


In [ ]:
from geopy.geocoders import Nominatim

# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Define the latitude and longitude
latitude = 40.7128
longitude = -74.0060

# Use the geolocator to reverse geocode the location
location = geolocator.reverse(f"{latitude}, {longitude}")

if location is not None:
    # Extract the address components from the location
    address = location.raw["address"]
    country = address.get("country", "")
    ocean = address.get("ocean", "")
    sea = address.get("sea", "")

    # Print the results
    if country:
        print(f"Country: {country}")
    if ocean:
        print(f"Ocean: {ocean}")
    if sea:
        print(f"Sea: {sea}")
else:
    print("Location not found.")


Country: United States


In [ ]:
from geopy.geocoders import Nominatim

# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Define the latitude and longitude
latitude = 40.7128
longitude = -74.0060

# Use the geolocator to reverse geocode the location
location = geolocator.reverse(f"{latitude}, {longitude}")

if location is not None:
    # Extract the address components from the location
    address = location.raw["address"]
    country = address.get("country", "")
    ocean = address.get("ocean", "")
    sea = address.get("sea", "")

    # Print the results
    if country:
        print(f"Country: {country}")
    if ocean:
        print(f"Ocean: {ocean}")
    if sea:
        print(f"Sea: {sea}")
else:
    print("Location not found.")


Country: United States


In [ ]:
type(latitude)

float

In [ ]:
type(latitude)

float